# ENERGY-AWARE ROUTE OPTIMIZATION FOR UAV- UGV SYSTEMS IN COLD STORAGE WAREHOUSES

Name - Anuda Wewalage <br/>
Index No - 211495N <br/>
Department - Transport Management & Logistics Engineering <br/>
Supervisor - Prof. Amila Thibbotuwawa <br/>
External Advisor : - Kasuni Weerasinghe <br/>

### Import Libraries

In [1]:
import networkx as nx
import pickle
from typing import List, Set, Tuple, Dict
# import numpy as np
# import matplotlib.pyplot as plt

from Algorithms import MEAStar, MACO, AStar
import Experiment as expt

## Import the Network
- This Network represent a two single selective rack on both sides of an isle.
- Each rack contains 32 pallet positions.
- Pallet positions in rack1 numbered 1 - 32 and rack 2 numbered 101 - 132
- 0 and 100 are positions that UGV can reach for deploying the drone.

In [2]:
def import_graph_pickle(filename='rack_network.gpickle'):
    """
    Import graph from pickle format.

    Args:
        filename: Input filename

    Returns:
        NetworkX graph
    """
    with open(filename, 'rb') as f:
        G = pickle.load(f)
    print(f"Graph loaded from {filename}")
    print(f"Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")
    return G

G_combined = import_graph_pickle("Intermediate Outputs/3d connected network.gpickle")
G_rack1 = import_graph_pickle("Intermediate Outputs/rack 1.gpickle")
G_rack2 = import_graph_pickle("Intermediate Outputs/rack 2.gpickle")

Graph loaded from Intermediate Outputs/3d connected network.gpickle
Nodes: 66, Edges: 444
Graph loaded from Intermediate Outputs/rack 1.gpickle
Nodes: 66, Edges: 444
Graph loaded from Intermediate Outputs/rack 2.gpickle
Nodes: 66, Edges: 444


## Experiment - Routing Algorithm

For the experiment, I create list of Random nodes and tested for 3 different scenarios.
1. for 4 nodes - usually for this level, Algorithm provide same output even run several times. So, we take data by running algorithm only once.
2. for 8 and 16 nodes - This level, algorithm gives different routes in multiple runs. So, we run optimal path algorithm once (since it is not a heuristic based algorithm) and other algorithms 50 times. Then take the average for the analyzis.

In [15]:
import pandas as pd
import time
import os
from datetime import datetime

In [4]:
# create objects
mea_star = MEAStar(G_combined)
a_star = AStar(G_combined)

In [ ]:
# Get user inputs
num_nodes = int(input("Enter number of nodes: "))
num_experiments = int(input("Enter number of experiments: "))

# Generate nodes once (outside the loop)
selected_nodes = expt.generate_unique_random_nodes_explicit(num_nodes)

# Initialize results list
results = []

#Check for old model (2d VRP) approach
start_time_ssp = time.time()
path1_ssp_tent, path2_ssp_tent, cost_ssp = expt.find_optimal_path_for_mixed_racks(selected_nodes, G_rack1, G_rack2)

time_ssp = time.time() - start_time_ssp

path1_ssp = [node for node in path1_ssp_tent if node in set(selected_nodes)]
path2_ssp = [node for node in path2_ssp_tent if node in set(selected_nodes)]

columns = ['expt_id', 'timestamp', 'selected_nodes', 'time_ssp', 'path1_ssp', 'path2_ssp', 'cost_ssp']

file_path = 'Output/Experiment_log.csv'

if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    # Determine next expt_id
    if not df.empty and 'expt_id' in df.columns:
        # Get the maximum existing expt_id and add 1
        last_id = df['expt_id'].max()
        next_id = int(last_id) + 1
    else:
        # Start from 1001 if file exists but no expt_id column or empty
        next_id = 1001
else:
    # Create new DataFrame starting from 1001
    df = pd.DataFrame(columns=columns)
    next_id = 1001

# Create new row
new_row = {
    'expt_id': next_id,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'selected_nodes': str(selected_nodes),
    'time_ssp': time_ssp,
    'path1_ssp': str(path1_ssp),
    'path2_ssp': str(path2_ssp),
    'cost_ssp': cost_ssp
}

# FIX: Convert to list of lists and create new DataFrame
# Convert existing df to list of dictionaries
if not df.empty:
    records = df.to_dict('records')
else:
    records = []

# Add new row
records.append(new_row)

# Create new DataFrame from all records
df = pd.DataFrame(records, columns=columns)

df.to_csv(file_path, index=False)

print(f"Experiment data added to {file_path} with expt_id: {next_id}")

# Main experiment loop
for experiment_no in range(1, num_experiments + 1):
    print(f"\n--- Running Experiment {experiment_no}/{num_experiments} ---")

    # ===== MEA_STAR Algorithm =====
    start_time_mea = time.time()

    # Build reduced graph with mea_star
    G_small_mea = expt.build_reduced_digraph(mea_star, selected_nodes)

    # Run MACO
    maco_mea = MACO(
        graph=G_small_mea,
        n_ants=6,
        n_iterations=200,
        alpha=1.0,
        beta=1.0,
        p=0.95
    )

    # Find optimal path
    path_maco_mea, cost_maco_mea = maco_mea.solve_tsp()

    time_maco_mea = time.time() - start_time_mea

    print(f"MEA_STAR - Cost: {cost_maco_mea:.4f}, Time: {time_maco_mea:.4f}s")

    # ===== A_STAR Algorithm =====
    start_time_astar = time.time()

    # Build reduced graph with a_star
    G_small_astar = expt.build_reduced_digraph(a_star, selected_nodes)

    # Run MACO
    maco_astar = MACO(
        graph=G_small_astar,
        n_ants=6,
        n_iterations=200,
        alpha=1.0,
        beta=1.0,
        p=0.95
    )

    # Find optimal path
    path_maco_astar, cost_maco_astar = maco_astar.solve_tsp()

    time_maco_astar = time.time() - start_time_astar

    print(f"A_STAR - Cost: {cost_maco_astar:.4f}, Time: {time_maco_astar:.4f}s")

    # ===== Record Results =====
    results.append({
        'experiment_no': experiment_no,
        'no_of_nodes': num_nodes,
        'node_list': selected_nodes,
        'cost_maco_mea': cost_maco_mea,
        'path_maco_mea': path_maco_mea,
        'time_maco_mea': time_maco_mea,
        'cost_maco_astar': cost_maco_astar,
        'path_maco_astar': path_maco_astar,
        'time_maco_astar': time_maco_astar
    })

# Create DataFrame
df_results = pd.DataFrame(results)

# Display results
print("\n" + "="*80)
print("EXPERIMENT RESULTS")
print("="*80)
print(df_results.to_string(index=False))

# Save to CSV
output_filename = f'Output/{next_id}_maco_experiments_{num_nodes}nodes_{num_experiments}runs.csv'
df_results.to_csv(output_filename, index=False)
print(f"\nResults saved to: {output_filename}")